Notes:
https://towardsdatascience.com/rfm-segmentation-in-e-commerce-e0209ce8fcf6 
https://medium.com/datasciencejunks/exploring-customers-segmentation-with-rfm-analysis-and-k-means-clustering-118f9ffcd9f0 
https://medium.com/web-mining-is688-spring-2021/using-k-means-to-segment-customers-based-on-rfm-variables-9d4d683688c8 


In [30]:
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans

In [7]:
raw_data = pd.read_csv('onlineretail.csv',encoding='unicode_escape')
raw_data.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,01-12-2010 08:26,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,01-12-2010 08:26,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,01-12-2010 08:26,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,01-12-2010 08:26,3.39,17850.0,United Kingdom


In [17]:
#checking for NULL
raw_data.isnull().sum()

InvoiceNo      0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
UnitPrice      0
CustomerID     0
Country        0
dtype: int64

In [13]:
#removing nulls/na
raw_data = raw_data.dropna(axis = 0, how = 'any')

In [16]:
# check duplicates
raw_data.duplicated().sum() # 5225 duplicated rows detected

0

In [15]:
# drop duplicates
raw_data = raw_data.drop_duplicates(ignore_index = True)

In [19]:
# convert datetime column format
raw_data.InvoiceDate = pd.to_datetime(raw_data.InvoiceDate, format='mixed')

In [20]:
# create trx_amount col
raw_data['TrxAmount'] = raw_data.Quantity * raw_data.UnitPrice

In [21]:
# include only positive TrxAmount values(exclude credit)
raw_data = raw_data[raw_data['TrxAmount'] > 0]

In [22]:
# cast CustomerID column to integer
raw_data.CustomerID = raw_data.CustomerID.astype('int')

In [23]:
# recency (r) df
data_r = raw_data[['InvoiceDate','CustomerID']].groupby('CustomerID')['InvoiceDate'].agg('max').reset_index()
data_r.head()

,CustomerID,InvoiceDate
0,12346,2011-01-18 10:01:00
1,12347,2011-10-31 12:25:00
2,12348,2011-09-25 13:13:00
3,12349,2011-11-21 09:51:00
4,12350,2011-02-02 16:01:00


In [24]:
# current time reference
cur_time = data_r.InvoiceDate.max()
# month diff function
def month_diff(cur_time, datetime_val): 
    return 12 * (cur_time.year - datetime_val.year) + (cur_time.month - datetime_val.month)
# recency month
data_r['Recency'] = data_r.InvoiceDate.apply(lambda datetime_val: month_diff(cur_time, datetime_val))

In [25]:
# clip max at 6 months backward
data_r.Recency = data_r.Recency.clip(lower = None, upper = 6)
# drop InvoiceDate column
data_r = data_r.drop('InvoiceDate', axis = 1)
# head
data_r.head()

,CustomerID,Recency
0,12346,6
1,12347,2
2,12348,3
3,12349,1
4,12350,6


In [26]:
# frequency & monetary(fm) df only w.r.t. last 6 months data
EARLIEST_DATE = pd.to_datetime('2011-06-09')
data_fm = raw_data[raw_data.InvoiceDate >= EARLIEST_DATE]
data_fm = data_fm[['InvoiceNo','CustomerID','TrxAmount']]
data_fm = data_fm.groupby('CustomerID')[['InvoiceNo','TrxAmount']].agg({'InvoiceNo':'nunique', 'TrxAmount':'mean'}).reset_index()
data_fm = data_fm.rename(columns = {'InvoiceNo':'Frequency', 'TrxAmount':'Monetary'})
data_fm.Monetary = data_fm.Monetary.round(2)
data_fm.head()

,CustomerID,Frequency,Monetary
0,12347,4,25.38
1,12348,1,103.33
2,12349,1,24.08
3,12352,2,19.77
4,12355,1,35.34


In [27]:
# join to have the final data df
data = data_r.merge(data_fm, on = 'CustomerID', how = 'left')
data = data.fillna(0)
data.head()

,CustomerID,Recency,Frequency,Monetary
0,12346,6,0.0,0.00
1,12347,2,4.0,25.38
2,12348,3,1.0,103.33
3,12349,1,1.0,24.08
4,12350,6,0.0,0.00


In [28]:
from sklearn.preprocessing import StandardScaler
# feature columns
feature_cols = ['Recency','Frequency','Monetary']
# standardized df for training
standardized_data = data.copy()
# standardization
scaler = StandardScaler()
scaler.fit(data[feature_cols])
standardized_features = scaler.transform(data[feature_cols])
standardized_data[feature_cols] = standardized_features

In [31]:
# run kmeans with the optimal k
kmeans = KMeans(n_clusters=3, init = 'k-means++')
kmeans.fit(standardized_data[feature_cols])
data['cluster'] = kmeans.labels_
data.head()

c:\Users\ERROL\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)


,CustomerID,Recency,Frequency,Monetary,cluster
0,12346,6,0.0,0.00,0
1,12347,2,4.0,25.38,1
2,12348,3,1.0,103.33,1
3,12349,1,1.0,24.08,1
4,12350,6,0.0,0.00,0


In [32]:
# look at the centroids per cluster
centroid_df = data.groupby('cluster')[['Recency', 'Frequency', 'Monetary','CustomerID']].agg({'Recency': 'mean', 'Frequency': 'mean', 'Monetary':'mean','CustomerID':'nunique'}).reset_index()
centroid_df = centroid_df.rename(columns = {'CustomerID':'NumBuyers'})
centroid_df

,cluster,Recency,Frequency,Monetary,NumBuyers
0,0,5.496530,0.582334,18.416669,1585
1,1,1.427326,3.435320,38.078140,2752
2,2,3.000000,1.000000,168469.600000,1
